In [1]:
from result_saver import SaverProvider
from Scratch import metadata_loader
provider = SaverProvider()

In [2]:
df = metadata_loader(extract=True)
#df = df[df.job_status != 'JobStatus.QUEUED']


In [3]:
for job_id in df.job_id:
    provider.retrieve_job(job_id)

13:21:47 Warning: Job ID cmxpnq65z3p0008dmqy0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider...
13:21:56 Warning: Job ID cmxpnq65z3p0008dmqy0 is still running. Aborting...
13:21:56 Warning: Job ID cmxpnpepbeb00088n6d0 not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GIT/Soft-Info/.Scratch/jobs. Retrieving it from the IBMQ provider...
13:22:05 Warning: Job ID cmxpnpepbeb00088n6d0 is still running. Aborting...
13:22:05 Warning: Job ID cmxpnmypbeb00088n6cg has 'JobStatus.ERROR'. Aborting...
13:22:05 Warning: Job ID cmxpnm6pbeb00088n6c0 has 'JobStatus.ERROR'. Aborting...
13:22:05 Warning: Job ID cmxphppbgfs0008763yg has 'JobStatus.ERROR'. Aborting...
13:22:05 Warning: Job ID cmxphny5z3p0008dmqsg has 'JobStatus.ERROR'. Aborting...
13:22:05 Warning: Job ID cmxph2cbgfs0008763xg not found in /Users/mha/My Drive/Desktop/Studium/Physik/Master/Semester 3/IBM/IBM GI

In [ ]:
df = df[df['job_id']=="cmxnz9dwjay0008cnwc0"]
if df.job_status.values == 'JobStatus.DONE':
    print("Job is done")    